In [1]:
import tensorflow as tf
import numpy as np
import os 
import time

In [2]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import torch.nn.functional as F

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [36]:
print(text[:400])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


In [6]:
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"

In [7]:
vocab = set(text)

In [8]:
char2idx = {ch:i for i,ch in enumerate(vocab)}
idx2char = {i:ch for i,ch in enumerate(vocab)}

In [9]:
# l1 = nn.Linear(3, 4)
# gru = nn.RNN(2, 3)
# input = torch.randn(5, 1, 2)

In [10]:
# outputs = gru(input)

In [11]:
# l1(outputs[0]).shape

In [12]:
class MyModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MyModel, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.embedding = nn.Embedding(input_size, input_size)
    self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=2)
    self.decoder = nn.Linear(hidden_size, output_size)
  def forward(self, x, hidden):
    embedding = self.embedding(x)
    outputs, h_n = self.rnn(embedding, hidden)
    outputs = self.decoder(outputs)
    return outputs, h_n.detach()

In [27]:
vocab_size = len(vocab)
hidden_size = 512
learning_rate = 0.0001
input_size = vocab_size
output_size = vocab_size
seq_length = 100
text_size = len(text)
EPOCHS = 15

In [15]:
inputs = [char2idx[i] for i in text[:seq_length]]
outputs = [char2idx[i] for i in text[1:seq_length+1]]

In [16]:
# text

In [17]:
enc_text = [char2idx[i] for i in text]
enc_text = torch.tensor(enc_text).long().to(device)
enc_text.unsqueeze_(1)
enc_text.shape

torch.Size([1115394, 1])

In [18]:
inputs = enc_text[:seq_length]
# inputs = inputs.unsqueeze(1)
inputs.shape

torch.Size([100, 1])

In [19]:
# model = MyModel(input_size, hidden_size, output_size).to(device)
# model(inputs)

In [20]:
# device

In [21]:
# outputs, h1 = model(inputs, None)

In [22]:
outputs.shape, h1.shape

(torch.Size([100, 1, 65]), torch.Size([2, 1, 512]))

In [23]:
# outputs, h1 = model(inputs, h1)

In [24]:
# outputs.shape, h1.shape

In [29]:
model = MyModel(input_size, hidden_size, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# LEARNING RATE = 0.0005

for epoch in range(EPOCHS):
  p = 0
  seq_len = seq_length
  total_loss = 0
  hidden = None
  cnt = 0
  while p+seq_len+1 < text_size:
    inputs = enc_text[p:p+seq_length]
    targets = enc_text[p+1:p+seq_length+1]
    outputs, hidden = model(inputs, hidden)
    loss = loss_fn(torch.squeeze(outputs), torch.squeeze(targets))
    total_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    cnt += 1
    p += seq_length
    if cnt % 500 == 0:
      print('|',end='')
  print('\n--------------------------')
      # print("Cnt : {} Loss : {:5f}".format(cnt, total_loss/cnt))
  print("Epoch : {} Epoch_loss : {:5f}".format(epoch+1, total_loss/cnt))
  total_loss = 0
  cnt = 0
  with torch.no_grad():
    rand_index = np.random.randint(text_size-1)
    input_seq = enc_text[rand_index:rand_index+1]
    print(idx2char[input_seq[0][0].item()], end='')
    hidden = None
    size = 0
    while size < 200:
      outputs, hidden = model(input_seq, hidden)
      outputs = F.softmax(torch.squeeze(outputs), dim=0)
      dist = Categorical(outputs)
      index = dist.sample()
      # input_seq[0][0] = index.item()
      input_seq[0][0] = index.item()
      # input = input.unsqueeze(0)
      print(idx2char[index.item()], end='')
      size += 1
    print("\n-----------------------")

||||||||||||||||||||||
--------------------------
Epoch : 1 Epoch_loss : 1.541408
gate.

ANTONO:
Did-gone not a wlother.

ANTONIO:
No down to the last of him, of your servants
And then woys would trink you bush, for exterminets mine exery
and hour myself bound
To wonders him, persfo
-----------------------
||||||||||||||||||||||
--------------------------
Epoch : 2 Epoch_loss : 1.328842
ags!

ANTONIO:
Welcommine!

ALINAN:
Who seem at your pleasure,
you that did--

GONZALO:
I long to you withal. But then he profisions
Beat volumeth any profutyastive; the
Clursed on your dear,
And but o
-----------------------
||||||||||||||||||||||
--------------------------
Epoch : 3 Epoch_loss : 1.269898
 Gtoll,
Who wores modest in the strivath;
Nembling vissarish the cholmours; the sighness, but knew of
Did know stolanching. Sweet sin; I am son;
Who master let'st a stooa livon.

ANTONIO:
We at a tendo
-----------------------
||||||||||||||||||||||
--------------------------
Epoch : 4 Epoch_loss : 1.2

In [30]:
#LEARNING RATE = 0.0001

for epoch in range(EPOCHS):
  p = 0
  seq_len = seq_length
  total_loss = 0
  hidden = None
  cnt = 0
  while p+seq_len+1 < text_size:
    inputs = enc_text[p:p+seq_length]
    targets = enc_text[p+1:p+seq_length+1]
    outputs, hidden = model(inputs, hidden)
    loss = loss_fn(torch.squeeze(outputs), torch.squeeze(targets))
    total_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    cnt += 1
    p += seq_length
    if cnt % 500 == 0:
      print('|',end='')
  print('\n--------------------------')
      # print("Cnt : {} Loss : {:5f}".format(cnt, total_loss/cnt))
  print("Epoch : {} Epoch_loss : {:5f}".format(epoch+1, total_loss/cnt))
  total_loss = 0
  cnt = 0
  with torch.no_grad():
    rand_index = np.random.randint(text_size-1)
    input_seq = enc_text[rand_index:rand_index+1]
    print(idx2char[input_seq[0][0].item()], end='')
    hidden = None
    size = 0
    while size < 300:
      outputs, hidden = model(input_seq, hidden)
      outputs = F.softmax(torch.squeeze(outputs), dim=0)
      dist = Categorical(outputs)
      index = dist.sample()
      # input_seq[0][0] = index.item()
      input_seq[0][0] = index.item()
      # input = input.unsqueeze(0)
      print(idx2char[index.item()], end='')
      size += 1
    print("\n-----------------------")

||||||||||||||||||||||
--------------------------
Epoch : 1 Epoch_loss : 1.826299
enqther.

GONZALO:
Your hold no into-tigond.

SEASRONA:
What think noted than the grate.

ANGELO:
Who will; nor wo do lovunin to his daughtedly man.
-it aN a plemuty josting athell.

MERANGIA:
Of your man dock, gy pointaning;
'bistechion of supperts the would not.

SEBASSIO:
Bilivigation my fachard:

-----------------------
||||||||||||||||||||||
--------------------------
Epoch : 2 Epoch_loss : 1.472017
hboot
Have so follow-desure upon thank thee?

GONZALO:
No,
Somether his food maliage she should, as such a son,
Whom your frof such never daughters; help, in love him:
Which is nothing give me, and long you to. Sir,
Abbidonal, solet warm'd.

SEBASTIAN:
It weak, my known'd the men all I being.

ANTONI
-----------------------
||||||||||||||||||||||
--------------------------
Epoch : 3 Epoch_loss : 1.360680
;
Khat thou never spoken and nither, fancy 'twixs
Would in.

GONZALO:
If would there thou art una fatc

In [35]:
total_loss = 0
cnt = 0
with torch.no_grad():
  rand_index = np.random.randint(text_size-1)
  input_seq = enc_text[rand_index:rand_index+1]
  print(idx2char[input_seq[0][0].item()], end='')
  hidden = None
  size = 0
  while size < 800:
    outputs, hidden = model(input_seq, hidden)
    outputs = F.softmax(torch.squeeze(outputs), dim=0)
    dist = Categorical(outputs)
    index = dist.sample()
    # input_seq[0][0] = index.item()
    input_seq[0][0] = index.item()
    # input = input.unsqueeze(0)
    print(idx2char[index.item()], end='')
    size += 1
  print("\n-----------------------")

ed:
Peace agaunt!

ANTONIO:
I told you
I'll have no means to the moon.

LUCENTIO:
Alack, for more; and then he had sound so
As old Angelo: Why the kind was own life
Should make thee bigger?

ANTONIO:
Well, Aage, a beggar-'I would wear my heart
verily of his never: advise you,
With this mad diem. Come, come to his cheruse
Berow.

ANTONIO:

ANTONIO:
He would not have been straight thinks it the men
tell sometimes of you.

RIVERS:
Both rise like words.

ANTONIO:
But when thou hast fine mercy?

GONZALO:
What! good Barnation, follow: mine.

ANTIONE:
The earth abrival be.

ANTONIO:
Call up succass! we were possess'd himself,
To lose her not; and if thy mainterary,
So often burst to-morrow,
Though turn'd my boltness as they do content
They then begins, And tell her son, and
from us thriving woman 
-----------------------
